### The notebook contains the pipeline for solving the NLP classification task via transformer-based methods.

                                                                                          Created by: Tsvigun Akim

In [ ]:
! git clone https://github.com/dheeraj7596/CONDA.git


Cloning into 'CONDA'...
remote: Enumerating objects: 189, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 189 (delta 13), reused 31 (delta 9), pack-reused 153
Receiving objects: 100% (189/189), 65.47 MiB | 24.34 MiB/s, done.
Resolving deltas: 100% (55/55), done.
Updating files: 100% (69/69), done.


In [ ]:
pip install transformers==4.20.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 110.8 MB/s eta 0:00:00


In [ ]:
pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 19.6 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import plotly.express as px

import transformers as ts
import datasets
import torch
from torch import nn

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [ ]:
# Upload the data
data = pd.read_csv('/content/CONDA/data/cls/imdb/train/train_13.csv')
data = data[['text', 'label']]
data

,text,label
0,"And maybe, as Fred Sandford used to say, ""one ...",negative
1,"The plot was very thin, although the idea of n...",negative
2,"""Pixote: A Lei do Mais Fraco"" deals with what ...",positive
3,Previous reviewer Claudio Carvalho gave a much...,positive
4,This is a typical late Universal Horror flick:...,negative
5,In the late 1940s there was a short film serie...,positive
6,"First of all, the actor they have to play Jesu...",negative
7,"When i got this movie free from my job, along ...",negative
8,do not ever watch this film...it is the bigges...,negative
9,This is a must for All but especially African ...,positive


In [ ]:
 data2 = pd.read_csv('/content/augmentation_generated.csv')
 data2 = data2[['Text','random_sentiment']].rename(columns={'Text': 'text', 'random_sentiment': 'label'})
 data2

,text,label
0,"This is a very sad film, but it is also one of...",negative
1,"""Pixote: A Lei do Mais Fraco"" is a wonderful m...",positive
2,"""Pixote: A Lei do Mais Fraco"" is a wonderful f...",positive
3,"""Pixote: A Lei do Mais Fraco"" is a wonderful f...",negative
4,"This is a beautiful film, but it is also one o...",positive
5,This movie is a must-see for many reasons. The...,positive
6,"This is a film that I have never seen before, ...",negative
7,This film is a classic example of a classic fi...,positive
8,This film is a must-see for anyone who is inte...,positive
9,This film is a must-see for anyone who is a fa...,negative


In [ ]:
df = pd.concat([data, data2])
df.head()

,text,label
0,"And maybe, as Fred Sandford used to say, ""one ...",negative
1,"The plot was very thin, although the idea of n...",negative
2,"""Pixote: A Lei do Mais Fraco"" deals with what ...",positive
3,Previous reviewer Claudio Carvalho gave a much...,positive
4,This is a typical late Universal Horror flick:...,negative


In [ ]:
df.loc[(df.label == 'negative'), 'label'] = 0
df.loc[(df.label == 'positive'), 'label'] = 1
df

,text,label
0,"And maybe, as Fred Sandford used to say, ""one ...",0
1,"The plot was very thin, although the idea of n...",0
2,"""Pixote: A Lei do Mais Fraco"" deals with what ...",1
3,Previous reviewer Claudio Carvalho gave a much...,1
4,This is a typical late Universal Horror flick:...,0
5,In the late 1940s there was a short film serie...,1
6,"First of all, the actor they have to play Jesu...",0
7,"When i got this movie free from my job, along ...",0
8,do not ever watch this film...it is the bigges...,0
9,This is a must for All but especially African ...,1


In [ ]:
id2label = {0: "negative", 1: "positive"}
label2id = {"negative": 0, "positive": 1}

In [ ]:
X = df.text
y = df.label

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
import datasets

# Define your data as dictionaries for each split
train_data = {
    "text": X_train,
    "label" : y_train

}

val_data = {
    "text": X_val,
    "label" : y_val
}


# Convert the dictionaries to `Dataset` objects
train_dataset = datasets.Dataset.from_dict(train_data)
val_dataset = datasets.Dataset.from_dict(val_data)

# Create a `DatasetDict` object containing the train, validation, and test sets
data= datasets.DatasetDict({
    "train": train_dataset,
    "validation": val_dataset,
})

# Print the sizes of the resulting datasets
print(data)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 32
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 15
    })
})


In [ ]:

train = data['train']
valid = data['validation']

In [ ]:
train[1]

{'text': '"Pixote: A Lei do Mais Fraco" is a wonderful movie, but it is also a terrible movie. It is a very sad movie, and it is very sad to see. In fact, it is one of the saddest movies I have ever seen.<br /><br />It is a sad film, because it is so sad and sad, and yet it is a beautiful movie, which is also sad because of its sadistic, sadistic and sadistic portrayal of the people who are living in this country. And it is sad because there are so many sad, sad things that are happening in the country, but also because of the sadistic nature of the country. In the end, the movie ends with a sad ending, and the people are left with nothing but sadness, sadness, and regret.<br/><br / />The movie is also very sad, because the people in the movie are so sad. The people who live in the movies are sad, but they are also sad. And they are sad because they have lost so much of their sense of self, and they are afraid of what they are going to see when they see the movie. The movie is sad, too

In [ ]:
# Upload the tokenizer
tokenizer = ts.AutoTokenizer.from_pretrained('distilbert-base-uncased', cache_dir='tmp/tokenizer')
tokenizer

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

PreTrainedTokenizerFast(name_or_path='distilbert-base-uncased', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [ ]:
def tokenizing_fn(instance):
    return tokenizer(instance['text'], truncation=True)

tokenized_train = data['train'].map(tokenizing_fn, batched=True)
tokenized_val = data['validation'].map(tokenizing_fn, batched=True)
tokenized_train

Map:   0%|          | 0/32 [00:00<?, ? examples/s]

Map:   0%|          | 0/15 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 32
})

In [ ]:
data_collator = ts.DataCollatorWithPadding(tokenizer = tokenizer)
data_collator

DataCollatorWithPadding(tokenizer=PreTrainedTokenizerFast(name_or_path='distilbert-base-uncased', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}), padding=True, max_length=None, pad_to_multiple_of=None, return_tensors='pt')

In [ ]:
accuracy_metric = datasets.load_metric('accuracy', cache_dir='tmp/metrics')
f1_metric = datasets.load_metric('f1', cache_dir='tmp/metrics')

cola_metric = datasets.load_metric('glue', 'cola', cache_dir='tmp/metrics')

<ipython-input-19-d84673391c6f>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  accuracy_metric = datasets.load_metric('accuracy', cache_dir='tmp/metrics')


In [ ]:
def compute_metrics(outputs):
    logits, labels = outputs
    preds = logits.argmax(1)

    metrics_dict = f1_metric.compute(
        references=labels, predictions=preds)
    metrics_dict.update(cola_metric.compute(
        references=labels, predictions=preds))

    return metrics_dict

In [ ]:

"""
def compute_metrics2(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)
    """

'\ndef compute_metrics2(eval_pred):\n    predictions, labels = eval_pred\n    predictions = np.argmax(predictions, axis=1)\n    return accuracy.compute(predictions=predictions, references=labels)\n    '

In [ ]:
model = ts.AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased",cache_dir = 'tmp/model', num_labels=2, id2label=id2label, label2id=label2id).to(device)


Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'pre_classi

In [ ]:
#model = ts.AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', cache_dir = 'tmp/model', num_labels = 2).to(device)

In [ ]:
training_args = ts.TrainingArguments(
    output_dir='tmp/model_output',
    # Batch size args
    num_train_epochs=50,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=150,
    # Optimizer args
    learning_rate=3e-5,
    weight_decay=1e-2,
    max_grad_norm=1.,
    # Scheduler args
    warmup_ratio=0.1,
    # Eval args
    metric_for_best_model='eval_f1',
    load_best_model_at_end=True,
    evaluation_strategy='epoch',
    logging_strategy='epoch',
    save_strategy='epoch',
    save_total_limit=3,
    # WANDB args
    report_to="wandb",  # enable logging to W&B
    gradient_accumulation_steps=8

)

In [ ]:
"""
training_args = ts.TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    warmup_ratio=0.1,
    # Eval args
    metric_for_best_model='eval_accuracy',
    load_best_model_at_end=True,
    evaluation_strategy='epoch',
    logging_strategy='epoch',
    save_strategy='epoch',
    save_total_limit=3,
    # WANDB args
    report_to="wandb",  # enable logging to W&B
    gradient_accumulation_steps=8
)
"""

PyTorch: setting up devices


In [ ]:
"""
training_args = ts.TrainingArguments(
    output_dir='tmp/model_output',
    # Batch size args
    num_train_epochs=40,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    # Optimizer args
    learning_rate=3e-5,
    weight_decay=1e-2,
    max_grad_norm=1.,
    # Scheduler args
    warmup_ratio=0.1,
    # Eval args
    metric_for_best_model='eval_accuracy',
    load_best_model_at_end=True,
    evaluation_strategy='epoch',
    logging_strategy='epoch',
    save_strategy='epoch',
    save_total_limit=3,
    # WANDB args
    report_to="wandb",  # enable logging to W&B
    gradient_accumulation_steps=8
)
callbacks = [ts.EarlyStoppingCallback(15)]
"""

In [ ]:
pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.1/205.1 kB 26.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.5 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=34ab441aac25a777e21ddea2a37ede6caff2cf8f570b0a23ff3ec1abdfc105bd
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


In [ ]:
trainer = ts.Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    #callbacks=callbacks,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 32
  Num Epochs = 50
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 256
  Gradient Accumulation steps = 8
  Total optimization steps = 50
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,F1,Matthews Correlation
1,0.089100,0.691433,0.000000,-0.277350
2,0.088600,0.696132,0.222222,-0.106600
3,0.086900,0.705748,0.421053,-0.377964
4,0.085000,0.720580,0.500000,0.000000
5,0.084200,0.739069,0.500000,0.000000
6,0.083700,0.761058,0.500000,0.000000
7,0.083400,0.778527,0.500000,0.000000
8,0.082300,0.785855,0.500000,0.000000
9,0.079500,0.783318,0.500000,0.000000
10,0.078400,0.777880,0.500000,0.000000


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 15
  Batch size = 150
Saving model checkpoint to tmp/model_output/checkpoint-1
Configuration saved in tmp/model_output/checkpoint-1/config.json
Model weights saved in tmp/model_output/checkpoint-1/pytorch_model.bin
tokenizer config file saved in tmp/model_output/checkpoint-1/tokenizer_config.json
Special tokens file saved in tmp/model_output/checkpoint-1/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****


TrainOutput(global_step=50, training_loss=0.04686973553150892, metrics={'train_runtime': 213.9668, 'train_samples_per_second': 7.478, 'train_steps_per_second': 0.234, 'total_flos': 211947837849600.0, 'train_loss': 0.04686973553150892, 'epoch': 50.0})

In [ ]:
from sklearn.metrics import classification_report, f1_score

In [ ]:
predictions = trainer.predict(tokenized_val)
predictions

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 15
  Batch size = 150


PredictionOutput(predictions=array([[-0.42062697, -0.03666979],
       [-0.45745122,  0.0577454 ],
       [-0.47409794, -0.02352801],
       [-0.3731865 , -0.15678103],
       [-0.45262668, -0.03058405],
       [-0.36216372, -0.10206576],
       [-0.5461439 ,  0.21389037],
       [-0.96196145,  0.3975346 ],
       [-0.36554655,  0.00176315],
       [-0.52260095, -0.01906837],
       [-0.3548543 ,  0.00321952],
       [-0.4524231 ,  0.03405854],
       [-0.6116942 ,  0.2538011 ],
       [-0.549585  ,  0.20467639],
       [-0.22738458, -0.26031148]], dtype=float32), label_ids=array([0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0]), metrics={'test_loss': 0.8326701521873474, 'test_f1': 0.5263157894736842, 'test_matthews_correlation': 0.18898223650461363, 'test_runtime': 0.3167, 'test_samples_per_second': 47.359, 'test_steps_per_second': 3.157})

In [ ]:
predicted_labels = predictions.predictions.argmax(-1)

In [ ]:
true_labels = predictions.label_ids

In [ ]:
print(classification_report(true_labels, predicted_labels), flush=True)
print("*" * 80, flush=True)
print("Micro f1", f1_score(true_labels, predicted_labels, average='micro'), flush=True)
print("Macro f1", f1_score(true_labels, predicted_labels, average='macro'), flush=True)

              precision    recall  f1-score   support

           0       1.00      0.10      0.18        10
           1       0.36      1.00      0.53         5

    accuracy                           0.40        15
   macro avg       0.68      0.55      0.35        15
weighted avg       0.79      0.40      0.30        15

********************************************************************************
Micro f1 0.4000000000000001
Macro f1 0.354066985645933
